# PRÁCTICA DE SIMULACIÓN - LAVANDERÍA DE CARROS
### Alejandro Enríquez 
### Ingeniería en Ciencias de la Computación
### 10mo Ciclo

Objetivo: en base al proyecto de lavandería de autos crear un proceso de salida de la máquina del vehículo que no interfiera con el uso normal de las máquinas

In [1]:
import simpy
import random
import json

In [27]:
max_vehiculos = 72
num_maquinas = 3
tiempo_lavado = 7
intervalo_llegada = 9
tiempo_simulacion = 40
tiempo_espera = 2
tiempo_traslado = 1

class Lavanderia(object):
    def __init__(self, environment, numMaquinas, tLavado):            
        self.env = environment
        self.maquinas = simpy.Resource(environment, numMaquinas)
        self.tLavado = tLavado
        
    def traslado(self, vehiculo):
        print('El vehículo %s empieza traslado a las %.2f.' % (vehiculo, env.now))
        yield self.env.timeout(random.randint(tiempo_traslado, tiempo_traslado +4))
        print('El vehículo %s entra a la máquina a las %.2f.' % (vehiculo, env.now))
        
    def lavar_vehiculo(self, vehiculo):
        yield self.env.timeout(random.randint(tiempo_lavado-2, tiempo_lavado+4))
        print("Suciedad removida al 100% del vehículo: " , vehiculo)
        
    def esperar(self, vehiculo):
        print('El vehículo %s sale de la máquina y entra a la sala de espera a las %.2f.' % (vehiculo, env.now))
        yield self.env.timeout(random.randint(tiempo_espera, tiempo_espera+3))
        print('El vehículo %s sale de la lavandería %.2f.' % (vehiculo, env.now))
        
def llegada_vehiculo(env, nombre, lavanderia):
    print('Llega vehiculo: %s a la hora %.2f' % (nombre, env.now))
    with lavanderia.maquinas.request() as maquina:
        yield maquina
        yield env.process(lavanderia.traslado(nombre))
        print('Entra a lavarse: %s a la hora %.2f. en la máquina' % (nombre, env.now))
        yield env.process(lavanderia.lavar_vehiculo(nombre))
        print("Vehiculo [%s] lavado a las %.2f." % (nombre, env.now))
        env.process(lavanderia.esperar(nombre))
               
def ejecutar_simulacion(env, nMaquinas, tLavado, intervalo):
    lavanderia = Lavanderia(env, nMaquinas, tLavado)
    for i in range(5):
        env.process(llegada_vehiculo(env, 'Vehiculo-%d' % (i+1), lavanderia))
        
    while True:
        yield env.timeout(random.randint(intervalo-2, intervalo+2))
        i+=1
        env.process(llegada_vehiculo(env, 'Vehiculo-%d' % (i+1), lavanderia))
        
print('Iniciando Lavandería...')
random.seed(77)

env = simpy.Environment()
env.process(ejecutar_simulacion(env, num_maquinas, tiempo_lavado, intervalo_llegada))

env.run(until=tiempo_simulacion)

#env.run(until = tiempo_simulacion)

Iniciando Lavandería...
Llega vehiculo: Vehiculo-1 a la hora 0.00
Llega vehiculo: Vehiculo-2 a la hora 0.00
Llega vehiculo: Vehiculo-3 a la hora 0.00
Llega vehiculo: Vehiculo-4 a la hora 0.00
Llega vehiculo: Vehiculo-5 a la hora 0.00
El vehículo Vehiculo-1 empieza traslado a las 0.00.
El vehículo Vehiculo-2 empieza traslado a las 0.00.
El vehículo Vehiculo-3 empieza traslado a las 0.00.
El vehículo Vehiculo-2 entra a la máquina a las 2.00.
El vehículo Vehiculo-3 entra a la máquina a las 2.00.
Entra a lavarse: Vehiculo-2 a la hora 2.00. en la máquina
Entra a lavarse: Vehiculo-3 a la hora 2.00. en la máquina
El vehículo Vehiculo-1 entra a la máquina a las 3.00.
Entra a lavarse: Vehiculo-1 a la hora 3.00. en la máquina
Suciedad removida al 100% del vehículo:  Vehiculo-2
Suciedad removida al 100% del vehículo:  Vehiculo-1
Vehiculo [Vehiculo-2] lavado a las 8.00.
El vehículo Vehiculo-2 sale de la máquina y entra a la sala de espera a las 8.00.
Vehiculo [Vehiculo-1] lavado a las 8.00.
El veh